In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
import seaborn
from sklearn.model_selection import train_test_split
seaborn.set(style='whitegrid'); seaborn.set_context('talk')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from sklearn.datasets import load_iris
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
import random

df = load_iris()
x = df.data  # we only take the first two features.
y = df.target
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=123)
print(len(x_train),len(x_test))

105 45


Q2 - MLP

In [ ]:
class MultiLayerPerceptron(BaseEstimator, ClassifierMixin):

    def __init__(self,epoch,lr):
      self.input = 4
      self.hidden = 4
      self.Output = 3  
      self.lr = lr
      self.epoch = epoch
      self.activation = self.sigmoid['activation']
      self.derivative = self.sigmoid['deriv']
      self.w_hidden = [[2 * random.random() - 1 for i in range(self.hidden)] for j in range(self.input)]
      self.w_output = [[2 * random.random() - 1 for i in range(self.Output)] for j in range(self.hidden)]
      self.bias = np.array([-1 for i in range(self.hidden)])
      self.bias_output = np.array([-1 for i in range(self.Output)])
      self.n_class = 3
    pass
    sigmoid = {'activation': lambda x: 1 / (1 + np.exp(-x)) ,'deriv': lambda x: x * (1 - x)}

    def Backpropagation(self, x):
        D_output = []
        erro_output = self.output - self.output_l2
        D_output = -1 * erro_output * self.derivative(self.output_l2)

        for i in range(self.hidden):
            for j in range(self.Output):
                self.w_output[i][j] -= self.lr * (D_output[j]
                        * self.output_l1[i])
                self.bias_output[j] -= self.lr * D_output[j]

        D_hidden = np.matmul(self.w_output, D_output) \
            * self.derivative(self.output_l1)

        for i in range(self.Output):
            for j in range(self.hidden):
                self.w_hidden[i][j] -= self.lr * (D_hidden[j]
                        * x[i])
                self.bias[j] -= self.lr * D_hidden[j]

    def predict(self, X, y):
        forecasts = []
        correct =0
        wrong=0
        forward = np.matmul(X, self.w_hidden) + self.bias
        forward = np.matmul(forward, self.w_output) + self.bias_output
        for i in forward:
            forecasts.append(max(enumerate(i), key=lambda x: x[1])[0])
        for i in range(len(forecasts)):
          if forecasts[i] == y[i]:
            correct+=1
        print("Accuracy: ",int(correct/len(forecasts)))
        return forecasts
        

    def fit(self, X, y):
        count_epoch = 1
        total_error = 0
        n = len(X)
        arr_epoch = []
        arr_err = []
        while count_epoch <= self.epoch:
            for (idx, inputs) in enumerate(X):
                self.output = np.zeros(self.n_class)
                self.output_l1 = self.activation(np.dot(inputs,
                        self.w_hidden) + self.bias.T)
                self.output_l2 = self.activation(np.dot(self.output_l1,
                        self.w_output) + self.bias_output.T)
                if y[idx] == 0:
                    self.output = np.array([1, 0, 0])  # Class 1 {1,0,0}
                elif y[idx] == 1:
                    self.output = np.array([0, 1, 0])  # Class 2 {0,1,0}
                elif y[idx] == 2:
                    self.output = np.array([0, 0, 1])  # Class 3 {0,0,1}

                square_error = 0
                for i in range(self.Output):
                    erro = (self.output[i] - self.output_l2[i]) ** 2
                    square_error = square_error + 0.05 * erro
                    total_error = total_error + square_error
                self.Backpropagation(inputs)

            total_error = total_error / n
            print ('Epoch ', count_epoch, '- Total Error: ',total_error)
            arr_err.append(total_error)
            arr_epoch.append(count_epoch)

            count_epoch += 1
        

        return self

In [ ]:
ETA = 0.1
EPOCHS = 15

model = MultiLayerPerceptron(EPOCHS,ETA)

model.fit(x_train, y_train)# Calling the function
model.predict(x_test,y_test)

Epoch  1 - Total Error:  0.06254078682980367
Epoch  2 - Total Error:  0.05490237250433753
Epoch  3 - Total Error:  0.04899814214215089
Epoch  4 - Total Error:  0.04414472750746622
Epoch  5 - Total Error:  0.04035932031866374
Epoch  6 - Total Error:  0.03748642238131732
Epoch  7 - Total Error:  0.035340116615529375
Epoch  8 - Total Error:  0.03373531797909455
Epoch  9 - Total Error:  0.032527854941211784
Epoch  10 - Total Error:  0.03161146178436908
Epoch  11 - Total Error:  0.03090859738424609
Epoch  12 - Total Error:  0.030363096610924158
Epoch  13 - Total Error:  0.029934272687816246
Epoch  14 - Total Error:  0.029591686451833365
Epoch  15 - Total Error:  0.029309528355681985
Accuracy:  0


[2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 2,
 2,
 2]

Q1 - Perceptron

In [ ]:
def predict(row,weights):
  activation = weights[0]
  for i in range(len(row)-1):
    activation += weights[i + 1] * row[i]
  return 1.0 if activation >= 0.0 else 0.0

def train_weights(x,y, l_rate, n_epoch):
  weights = [0.0 for i in range(len(x[0]))]
  
  for epoch in range(n_epoch):
    sum_error = 0.0
    j=0
    for row in x:
      prediction = predict(row, weights)
      error = y[j] - prediction
      sum_error += error**2
      weights[0] = weights[0] + l_rate * abs(error)
      j+=1
      for i in range(len(row)-1):
        weights[i + 1] = weights[i + 1] + l_rate * abs(error) * row[i]
    accuracy = int(sum_error)/len(x)  
    print('>epoch=%d, lrate=%.3f, accuracy=%.3f' % (epoch, l_rate, accuracy))
    
  return weights

l_rate = 0.09
n_epoch = 15
weights = train_weights(x_train, y_train, l_rate, n_epoch)


In [ ]:
class Perceptron(BaseEstimator, ClassifierMixin):

    def __init__(self,epoch,lr):
      self.input = 4
      self.hiddenLayer = 4
      self.OutputLayer = 3  
      self.lr = lr
      self.epoch = epoch
      self.BiasHiddenValue = -1  
      self.BiasOutputValue = -1 
      self.activation = self._activation['sigmoid']  # Activation function
      self.derivative = self._deriv['sigmoid']
      self.w_hidden = [[2 * random.random() - 1 for i in range(self.hiddenLayer)] for j in range(self.input)]
      self.w_output = [[2 * random.random() - 1 for i in range(self.OutputLayer)] for j in range(self.hiddenLayer2)]
      self.bias_hidden = np.array([self.BiasHiddenValue for i in range(self.hiddenLayer)])
      self.bias_output = np.array([self.BiasOutputValue for i in range(self.OutputLayer)])
      self.n_class = 3
    pass
    _activation = {'sigmoid': lambda x: 1 / (1 + np.exp(-x)),'tanh': lambda x: np.tanh(x), 'Relu': lambda x: x * (x> 0)}
    _deriv = {'sigmoid': lambda x: x * (1 - x), 'tanh': lambda x: 1 / (- x ** 2), 'Relu': lambda x: 1 * (x > 0)}

    def Backpropagation(self, x):
        D_output1 = []
        D_output2 = []
        erro_output = self.output_l3 - self.output_l2
        D_output1 = -1 * erro_output * self.derivative(self.output_l2)
        erro_output = self.output - self.output_l3
        D_output2 = -1 * erro_output * self.derivative(self.output_l3)

        for i in range(self.hiddenLayer1):
            for j in range(self.hiddenLayer2):
                self.w_hidden2[i][j] -= self.lr * (D_output1[j]* self.output_l1[i])
                self.bias_hidden2[j] -= self.lr * D_output1[j]
        for i in range(self.hiddenLayer2):
            for j in range(self.OutputLayer):
                self.w_output[i][j] -= self.lr * (D_output2[j]* self.D_output2[i])
                self.bias_output[j] -= self.lr * D_output2[j]

        D_hidden = np.matmul(self.w_output, D_output1) * self.derivative(self.output_l1)

        for i in range(self.OutputLayer):
            for j in range(self.hiddenLayer):
                self.w_hidden[i][j] -= self.lr * (D_hidden[j]
                        * x[i])
                self.bias_hidden[j] -= self.lr * D_hidden[j]

    def predict(self, X, y):
        '''Returns the predictions for every element of X'''

        forecasts = []
        forward = np.matmul(X, self.w_hidden) + self.bias_hidden
        forward = np.matmul(forward, self.w_output) + self.bias_output

        for i in forward:
            forecasts.append(max(enumerate(i), key=lambda x: x[1])[0])

        print(' Number of Sample  | Class |  Output |  Hoped Output')
        for i in range(len(forecasts)):
            if forecasts[i] == 0:
                print (
                    ' id :',
                    i,
                    '          | Iris-Setosa        |  Output:',
                    forecasts[i],
                    '   |',
                    y[i],
                    )
            elif forecasts[i] == 1:
                print (
                    ' id :',
                    i,
                    '          | Iris-Versicolour   |  Output:',
                    forecasts[i],
                    '   |',
                    y[i],
                    )
            elif forecasts[i] == 2:
                print (
                    ' id :',
                    i,
                    '          | Iris-Virginica     |  Output:',
                    forecasts[i],
                    '   |',
                    y[i],
                    )

        return forecasts
        pass

    def fit(self, X, y):
        count_epoch = 1
        total_error = 0
        n = len(X)
        arr_epoch = []
        arr_err = []
        while count_epoch <= self.epoch:
            for (idx, inputs) in enumerate(X):
                self.output = np.zeros(self.n_class)
                self.output_l1 = self.activation(np.dot(inputs,
                        self.w_hidden1) + self.bias_hidden1.T)
                self.output_l2 = self.activation(np.dot(self.output_l1,
                        self.w_hidden2) + self.bias_hidden2.T)
                self.output_l3 = self.activation(np.dot(self.output_l2,
                        self.w_output) + self.bias_output.T)
                if y[idx] == 0:
                    self.output = np.array([1, 0, 0])  # Class 1 {1,0,0}
                elif y[idx] == 1:
                    self.output = np.array([0, 1, 0])  # Class 2 {0,1,0}
                elif y[idx] == 2:
                    self.output = np.array([0, 0, 1])  # Class 3 {0,0,1}

                square_error = 0
                for i in range(self.OutputLayer):
                    erro = (self.output[i] - self.output_l3[i]) ** 2
                    square_error = square_error + 0.05 * erro
                    total_error = total_error + square_error

                self.Backpropagation(inputs)

            total_error = total_error / n
            print ('Epoch ', count_epoch, '- Total Error: ',total_error)
            arr_err.append(total_error)
            arr_epoch.append(count_epoch)

            count_epoch += 1
        

        return self

In [ ]:
ETA = 0.1
EPOCHS = 10

model = Perceptron(EPOCHS,ETA)

model.fit(x_train, y_train)# Calling the function
model.predict(x_test,y_test)